In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sjögrens_Syndrome"
cohort = "GSE40611"

# Input paths
in_trait_dir = "../../input/GEO/Sjögrens_Syndrome"
in_cohort_dir = "../../input/GEO/Sjögrens_Syndrome/GSE40611"

# Output paths
out_data_file = "../../output/preprocess/Sjögrens_Syndrome/GSE40611.csv"
out_gene_data_file = "../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE40611.csv"
out_clinical_data_file = "../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE40611.csv"
json_path = "../../output/preprocess/Sjögrens_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression data of parotid tissue from Primary Sjogren’s Syndrome and controls"
!Series_summary	"Primary Sjögren’s syndrome (pSS) is a chronic autoimmune disease with complex etiopathogenesis. Here we use Affymetrix U133 plus 2.0 microarray gene expression data from human parotid tissue. Parotid gland tissues were harvested from 17 pSS and 14 14 non-pSS sicca patients and 18 controls. The data were used in the following article:  Nazmul-Hossain ANM, Pollard RPE, Kroese FGM, Vissink A, Kallenberg CGM, Spijkervet FKL, Bootsma H, Michie SA, Gorr SU, Peck AB, Cai C, Zhou H, Horvath S, Wong DTW (2012) Systems Analysis of Primary Sjögren’s Syndrome Pathogenesis in Salivary Glands: Comparative Pathways and Molecular Events in Humans and a Mouse Model."
!Series_overall_design	"Parotid gland tissues were harvested from 17 pSS and 14 non-pSS sicca patients and 18 controls."
Sample Characteristics Dictionary:
{0: ['disease status: Control', 'disease sta

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# This dataset appears to be an Affymetrix gene expression microarray dataset (U133 plus 2.0)
# which contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary, we can see:
# - trait_row: 0 (disease status: Control, pSS, Sicca)
# - age_row: None (not available in the dictionary)
# - gender_row: None (not available in the dictionary)
trait_row = 0
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait values to binary (0: Control/Sicca, 1: pSS)"""
    if value is None:
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Map values to binary
    if value.lower() == 'pss':
        return 1  # Primary Sjögren's syndrome
    elif value.lower() in ['control', 'sicca']:
        return 0  # Control or non-pSS sicca patients
    else:
        return None

# Since age and gender data are not available, we'll define placeholder functions
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We only proceed if trait data is available
if trait_row is not None:
    # Check if clinical_data exists in the global scope
    # This assumes clinical_data was loaded in a previous step
    if 'clinical_data' in globals():
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(clinical_features)
        print("Clinical Features Preview:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print("Error: clinical_data not found. Make sure it was loaded in a previous step.")
else:
    print("Skipping clinical feature extraction as trait data is not available.")


Clinical Features Preview:
{'GSM997850': [0.0], 'GSM997851': [0.0], 'GSM997852': [0.0], 'GSM997853': [0.0], 'GSM997854': [0.0], 'GSM997855': [0.0], 'GSM997856': [0.0], 'GSM997857': [0.0], 'GSM997858': [0.0], 'GSM997859': [0.0], 'GSM997860': [0.0], 'GSM997861': [0.0], 'GSM997862': [0.0], 'GSM997863': [0.0], 'GSM997864': [0.0], 'GSM997865': [0.0], 'GSM997866': [0.0], 'GSM997867': [0.0], 'GSM997868': [1.0], 'GSM997869': [1.0], 'GSM997870': [1.0], 'GSM997871': [1.0], 'GSM997872': [1.0], 'GSM997873': [1.0], 'GSM997874': [1.0], 'GSM997875': [1.0], 'GSM997876': [1.0], 'GSM997877': [1.0], 'GSM997878': [0.0], 'GSM997879': [0.0], 'GSM997880': [0.0], 'GSM997881': [0.0], 'GSM997882': [0.0], 'GSM997883': [0.0], 'GSM997884': [0.0], 'GSM997885': [0.0], 'GSM997886': [0.0], 'GSM997887': [0.0], 'GSM997888': [0.0], 'GSM997889': [0.0], 'GSM997890': [0.0], 'GSM997891': [0.0], 'GSM997892': [1.0], 'GSM997893': [1.0], 'GSM997894': [1.0], 'GSM997895': [1.0], 'GSM997896': [1.0], 'GSM997897': [1.0], 'GSM997898':

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers shown from the previous step
# These identifiers follow Affymetrix probe ID format (e.g., '1007_s_at', '1053_at')
# They are not standard human gene symbols (like BRCA1, TP53, etc.)
# Affymetrix probe IDs need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe gene identifiers in expression data and annotation data
# From the previous outputs we can see:
# - In gene_data, the index is ID (e.g., '1007_s_at')
# - In gene_annotation, there are columns 'ID' and 'Gene Symbol'

# 2. Extract gene mapping dataframe with probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply gene mapping to convert from probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the number of mapped genes to check
print(f"Number of genes after mapping: {len(gene_data)}")

# Preview the first few rows of the gene expression data
print("Gene expression data preview (first 5 genes):")
print(gene_data.head())


Number of genes after mapping: 21278
Gene expression data preview (first 5 genes):
          GSM997850   GSM997851   GSM997852   GSM997853   GSM997854  \
Gene                                                                  
A1BG       200.7886   264.12790   222.02390  1130.58000   115.61810   
A1BG-AS1   348.5785    75.04438    17.70408   348.76770    53.74231   
A1CF       376.0515  1038.22820   218.99734  1637.57586   509.34959   
A2M        999.7460  3896.31248  5243.59760  1262.17840  3060.09980   
A2M-AS1    136.3005   438.83670   282.71130   522.03900   876.44090   

           GSM997855  GSM997856   GSM997857  GSM997858   GSM997859  ...  \
Gene                                                                ...   
A1BG        40.41677  1250.6850   243.05170   315.2728    13.07202  ...   
A1BG-AS1    27.96156   422.7090    69.14751   283.7576   104.83430  ...   
A1CF       746.00250  1613.7927   537.44098  1199.0763   168.77953  ...   
A2M       6499.39390   837.3244  4695.02140 

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data shape: {clinical_df.shape}")
print(clinical_df.head())

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from CD4 T-cells of pSS patients and healthy controls."
)

# 7. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (19845, 49)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE40611.csv
Loaded clinical data shape: (1, 49)
                   GSM997850  GSM997851  GSM997852  GSM997853  GSM997854  \
Sjögrens_Syndrome        0.0        0.0        0.0        0.0        0.0   

                   GSM997855  GSM997856  GSM997857  GSM997858  GSM997859  ...  \
Sjögrens_Syndrome        0.0        0.0        0.0        0.0        0.0  ...   

                   GSM997889  GSM997890  GSM997891  GSM997892  GSM997893  \
Sjögrens_Syndrome        0.0        0.0        0.0        1.0        1.0   

                   GSM997894  GSM997895  GSM997896  GSM997897  GSM997898  
Sjögrens_Syndrome        1.0        1.0        1.0        1.0        1.0  

[1 rows x 49 columns]
Linked data shape: (49, 19846)
           Sjögrens_Syndrome       A1BG   A1BG-AS1        A1CF         A2M  \
GSM997850                0.0   200.7886  348.57850   376.05150   999.74600   
GSM997851                0.0   264.1279 

Shape after handling missing values: (49, 19846)
For the feature 'Sjögrens_Syndrome', the least common label is '1.0' with 17 occurrences. This represents 34.69% of the dataset.
The distribution of the feature 'Sjögrens_Syndrome' in this dataset is fine.



Linked data saved to ../../output/preprocess/Sjögrens_Syndrome/GSE40611.csv
